# About objects and classes

Modern C++ does not revolutionize the object-oriented features but rather brings many small improvements for most of the identified lacks.

## Improvement of object constructions

#### Delegating constructors

In [ ]:
#include <iostream>

In [ ]:
struct ClassA {
    ClassA() : ClassA(0) { m_default = true ; }
    ClassA( int a_x ) : m_x {a_x} { m_default = false ; }
    int m_x ;
    bool m_default ;
} ;

In [ ]:
void display( ClassA const & a_obj )
{
  if (a_obj.m_default) { std::cout<<"(default)"<<a_obj.m_x<<std::endl ; }
  else { std::cout<<"(explicit)"<<a_obj.m_x<<std::endl ; }
}

In [ ]:
ClassA a1 {}, a2 {0} ;
display(a1) ;
display(a2) ;

#### Initialization of data members

In [ ]:
struct ClassB {
  ClassB() {} ;
  ClassB( int a_x ) : m_x {a_x} {}
  int m_x { 0 } ;
} ;

In [ ]:
ClassB b0 {} ;
ClassB b1 {1} ;
std::cout << b0.m_x << std::endl ;
std::cout << b1.m_x << std::endl ;

#### Inheritance and constructors

In [ ]:
#include <iostream>

In [ ]:
struct ClassA {
    ClassA( int ) { std::cout << "ClassA(int)" << std::endl ; }
    ClassA( int, int ) { std::cout << "ClassA(int,int)" << std::endl ; }
} ;

In [ ]:
struct ClassB : public ClassA {
  using ClassA::ClassA ; // either A::A( int) or A::A( int, int ) can be used
                         // as if they were declared as B::B( int) and B::B( int, int )
} ;

In [ ]:
ClassB b1 {1} ;
ClassB b2 {1,2} ;

## Improvement of declaration of member functions

#### Forbid a function

In [ ]:
// class cannot be copied in C++03
class no_copies {
  public:
    no_copies() {}
  private:
    no_copies( const no_copies & ) ;
    no_copies & operator=( const no_copies & ) ;
} ;

In [ ]:
// class cannot be copied in C++11
class no_copies_v2 {
  public:
    no_copies_v2() {}
    no_copies_v2( no_copies_v2 const & ) = delete ;
    no_copies_v2 & operator=( no_copies_v2 const & ) = delete ;
} ;

#### We can now prevent certain implicit conversions

In [ ]:
struct FooStruct {
  void foo_method(short) {}
  void foo_method(int) = delete ;
} ;

In [ ]:
FooStruct s ;
s.foo_method(42) ; // Error, int overload declared deleted
s.foo_method(static_cast<short>(42)) ; // OK


#### Modify the signature but keep the default implementation

In [ ]:
class Y {
  public:
    Y & operator=( Y const & ) = default ; // Make it explicit
    virtual ~Y() = default ; // Add virtual
  protected:
    Y() = default ; // Change access
} ; 

## Hiding is still an issue

When one use both inheritance and overloading, i.e. multiple functions with the same name but different signatures (the number or type of arguments), how does it work?

In [ ]:
struct Base
 {
  void display( int ) { std::cout<<"Base::display( int )"<<std::endl ; }
  void display( float ) { std::cout<<"Base::display( float )"<<std::endl ; }
 } ;

In [ ]:
struct Derived : public Base
 {
  void display( float ) { std::cout<<"Derived::display( float )"<<std::endl ; }
  void display( double ) { std::cout<<"Derived::display( double )"<<std::endl ; }
 } ;

In [ ]:
void check( Derived obj )
 { 
  obj.display(42) ;
  obj.display(3.14f) ;
  obj.display(3.14) ;
 }

When compiling `obj.mf(...)`, the compiler follow those steps:
1. search some member function named `mf` in the class of `obj`;
2. while not found at least one `mf`, move to its base classes, one after the other;
3. in the selected class, within all overloaded `mf` functions, select the one which fit better the call arguments `(...)`.

### Good old-fashioned practice

When you redefine an inherited member function, **redefine all the base class functions which share the same name**, or you will hide some of them.

## Improvement of virtual member functions

#### Control the redefinition of virtual methods

In [ ]:
class ClassA
 {
  public :
    virtual void fct1() =0 ;
    virtual void fct2( int ) =0 ;
    virtual void fct3( bool ) =0 ;
 } ;

In [ ]:
class ClassB : public ClassA
 {
  public :
    void fct1() override ;       // OK
    void ft2( int ) override ;   // error: A::ft2 does not exist
    void fct2( bool ) override ; // error: not the good types
 } ;

#### Forbid redefining virtual methods and abstract classes

In [ ]:
struct ClassA {
    virtual void fct1() =0 ;
    virtual void fct2( int ) =0 ;
} ;

In [ ]:
struct ClassB : public ClassA {
    virtual void fct3( bool ) final ;
} ;

In [ ]:
struct ClassC final : public ClassB {
    void fct1() override ;       // OK
    void fct3( bool ) override ; // error: B::fct3 is final
} ;

In [ ]:
struct ClassD : public ClassC {} ;               // error: C is final         

#### Reminder: make the destructor virtual

Indeed, you may create a derived object with `new` and store its address in a base class pointer. When calling `delete` on this pointer, one will bypass the derived destructor, unless it is virtual.

In [ ]:
class ClassA
 {
  public :
    virtual void fct1() =0 ;
    virtual void fct2( int ) =0 ;
    virtual void fct3( bool ) =0 ;
    virtual ~ClassA() {}
 } ;

#### Reminder: avoid virtual functions at low level

Because of the cost, virtual functions are rather used for large and not numerous objects: the upper software layers of your application. For what concerns the numerous and small objects, it is preferable to use templates (which slow down the compilation but have no effect during execution).

## Reminder about function-objects

If a class supply an `operator()`, its objects can behave similarly to a function. The member variables of the class can be seen as extraneous parameters for this function.

In [ ]:
#include <iostream>

In [ ]:
class LinearFunction
 {
  public :
    LinearFunction( double constant ) : m_constant(constant) {}
    double operator()( double value ) { return m_constant*value ; }
  private :
    double m_constant ;    
 } ;

In [ ]:
LinearFunction times2(2) ;
const int SIZE = 10 ;
double values[SIZE] = { 0, 1, 2, 3, 4, 5, 6, 7, 8, 9 } ;
for ( int i=0 ; i<SIZE ; ++i )
 { std::cout << times2(values[i]) << " " ; }

### The recommended use of function-objects

This type of classes is especially used with algorithms of the standard library.

Below, we review the previous function-object, in order to integrate the call to `std::cout`, and then we combine it with `std::foreach`.

In [ ]:
#include <iostream>

In [ ]:
class LinearFunctionPrint {
  public :
    LinearFunctionPrint( double constant ) : m_constant(constant) {}
    void operator()( double value )
     { std::cout << (m_constant*value) << " " ; }
  private :
    double m_constant ;    
 } ;

In [ ]:
const int SIZE = 10 ;
double values[SIZE] = { 0, 1, 2, 3, 4, 5, 6, 7, 8, 9 } ;

In [ ]:
#include <algorithm>

In [ ]:
LinearFunctionPrint times2cout(2) ;
std::for_each(values,values+SIZE,times2cout) ;

In [ ]:
std::for_each(values,values+SIZE,LinearFunctionPrint(2)) ;

Objet-functions are classes. They can also use inheritance:

In [ ]:
class LinearFunctionPrint : public LinearFunction
 {
  public :
    LinearFunctionPrint( double constant ) : LinearFunction(constant) {}
    void operator()( double value )
     { std::cout << LinearFunction::operator()(value) << " " ; }
 } ;

In [ ]:
std::for_each(values,values+SIZE,LinearFunctionPrint(2)) ;

# Take away

- The new `delete` keyword forbids more explicitly some functions.
- The new `override` keyword helps to detect typos when redefining inherited member functions.
- Still, be aware of the **hiding pitfall** if you overload with different signatures.
- Avoid the costly virtual functions for low level objects.
- Function-objects can incorporate parameters and be passed on to algorithms.

# Questions?

# Exercise

In the code below:
- insert one `= delete`, one `= default` and one `override` ;
- just in case, give default values to the members variables ;
- prevent the compiler from implicitly transforming a double into a particle.
- what is still lacking?

In [ ]:
%%file tmp.objets.cpp

#include <cstdlib> // pour std::rand()
#include <iostream>
#include <string>

class Particle
 {
  public  :
    Particle( double a_mass ) : m_mass {a_mass} {}
    double mass() { return m_mass ; }
    virtual std::string name() { return "Particle" ; }
    ~Particle() {}
  private  :
    Particle( Particle const & ) ; // non copiable
    double m_mass ;
 } ;

class ChargedParticle : public Particle
 {
  public  :
    ChargedParticle( double a_mass, double a_charge )
     : Particle(a_mass), m_charge {a_charge} {}
    double charge() { return m_charge ; }
    virtual std::string name() { return "ChargedParticle" ; }
  private  :
    double m_charge ;
 } ;

void print( Particle & a_p  )
 {
  std::cout << a_p.name() << std::endl ;          
  std::cout << "  mass = " << p.mass() << std::endl ;         
 }

int main()
 {
  for ( int i = 0 ; i < 5 ; ++i )
   {
    if ( std::rand() < (.5*RAND_MAX) )
     {
      Particle p {2} ;
      print(p) ;
     }
    else
     {
      ChargedParticle p {1,1} ;
      print(p) ;
      std::cout << "  charge = " << p.charge() << std::endl ;         
     }
   }
 }

In [ ]:
!rm -f tmp.objets.exe && g++ -std=c++17 tmp.objets.cpp -o tmp.objets.exe

In [ ]:
!./tmp.objets.exe

© *CNRS 2024*  
*This document was created by David Chamont and translated by Olga Abramkina. It is available under the [License Creative Commons - Attribution - No commercial use - Shared under the conditions 4.0 International](http://creativecommons.org/licenses/by-nc-sa/4.0/)*